# Import and Set Paths

In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import statistics 
from scipy.stats import skew
import numpy as np
import pandas as pd
from google.oauth2 import service_account #Control API Keys
from google.cloud import vision
from pdf2image import convert_from_path #Libary convert pdf file to img file
import os
import io
import random
from random import randrange
from IPython.display import Image
from shapely.geometry import Polygon
import argparse
from enum import Enum
import io
from google.cloud.vision import types
from PIL import Image, ImageDraw
from PIL import ImageFont
from pyresparser import ResumeParser
import shutil
from shutil import copyfile
import random
import spacy
nlp = spacy.load("en_core_web_sm")

In [3]:
doc = '/Users/kunal/Documents/VdartResumeProject/VisionAPi/Document_402.pdf'
docPath = '/Users/kunal/Documents/VdartResumeProject/VisionAPi/'
imgTxtVisionAPIPath = "/Users/kunal/Documents/VdartResumeProject/APIKEYSGOOGLE/resumeMatcher-pdf2img.json"
pdfIMGPopplerPath = '/Users/kunal/Documents/VdartResumeProject/Poppler/poppler-0.68.0_x86/poppler-0.68.0/bin/'
fontPath = '/Users/kunal/Documents/VdartResumeProject/Font/FreeMonoBold.ttf'
allResumesPath = "/Users/kunal/Documents/VdartResumeProject/50_resumes/"
finalSaveImg = '/Users/kunal/Documents/VdartResumeProject/VisionAPi/ALLFINALSAVEDIMG'

# Functions

In [4]:
def convert_pdf_2_image(uploaded_image_path, uploaded_image):
    #Using the convert_from_path function
    #Same name as pdf but converted to img
    #Watch out for poppler -- necceasary to function
    os.chdir(uploaded_image_path) # Change the working diretory to path that contains the PDF file
    file_name = str(uploaded_image).replace('.pdf','') # file name for png still going to get changed later
    pages = convert_from_path(uploaded_image, 200,poppler_path=pdfIMGPopplerPath) #function to change pdf to img
    pageNumCount = 1 #numbering for all the different images if pdf is multiple pages
    outputNames = []
    for page in pages:
        output_file = file_name+"_"+str(pageNumCount) + '.jpg'#uptaded name for image
        page.save(output_file, 'JPEG')#save img
        pageNumCount +=1
        outputNames.append(output_file)
    return outputNames #names of img

In [5]:
def collectAll(fontPath, imagePath, resumePDFPath, finalSaveImg):
    with io.open(imagePath, 'rb') as image_file:
        content = image_file.read()
    image = types.Image(content=content)
    response = client.document_text_detection(image=image)
    document = response.full_text_annotation
    #Document, imagePath, resumePDFPath
    df = createExcelData(imagePath, document)
    boundBoxAll = getBoundingBoxes(imagePath, document)
    image = Image.open(imagePath)
    SINGLEOUTLINEBOXIMG = draw_single_box(image, boundBoxAll[randrange(3)][0])
    SINGLETRANSLUCENTBOXIMG = drawsingleBoxVertice(imagePath, (0,0,255), 0.25, boundBoxAll[randrange(3)][0])
    ALLOBJECTSBOXIMG = colorALL(imagePath, boundBoxAll)
    ALLSKILLSFROMRandomWords = colorAllSkills(imagePath, findRandomWordsFromDocument(document), document)
    ALLENTITIESAUTOCOLORDOCUMENTIMG = automaticColoringOfEntitiesDocument(imagePath, resumePDFPath, document)
    ALLDATESSPACYIMG = SpacyDatesColor(document,imagePath)
    ALLBIGWORDS = highlightBigWords(document,imagePath)
    ALLCAPITALBIGWORDS = capitalLargeWords(document,imagePath)
    width = 6616
    height = 4678
    #1654, 2339
    cols = 4
    rows = 2
    thumbnail_width = width//cols
    thumbnail_height = height//rows
    size = thumbnail_width, thumbnail_height
    new_im = Image.new('RGB', (width, height))
    ims = []
    SINGLEOUTLINEBOXIMG.thumbnail(size)
    ims.append(SINGLEOUTLINEBOXIMG)
    SINGLETRANSLUCENTBOXIMG.thumbnail(size)
    ims.append(SINGLETRANSLUCENTBOXIMG)
    ALLOBJECTSBOXIMG.thumbnail(size)
    ims.append(ALLOBJECTSBOXIMG)
    ALLSKILLSFROMRandomWords.thumbnail(size)
    ims.append(ALLSKILLSFROMRandomWords)
    ALLENTITIESAUTOCOLORDOCUMENTIMG.thumbnail(size)
    ims.append(ALLENTITIESAUTOCOLORDOCUMENTIMG)
    ALLDATESSPACYIMG.thumbnail(size)
    ims.append(ALLDATESSPACYIMG)
    ALLBIGWORDS.thumbnail(size)
    ims.append(ALLBIGWORDS)
    ALLCAPITALBIGWORDS.thumbnail(size)
    ims.append(ALLCAPITALBIGWORDS)
    i = 0
    x = 0
    y = 0
    for col in range(cols):
        for row in range(rows):
            #print(i, x, y)
            new_im.paste(ims[i], (x, y))
            i += 1
            y += thumbnail_height
        x += thumbnail_width
        y = 0
    os.chdir(finalSaveImg)
    fileName = os.path.basename(imagePath)[:-4]+".jpg"
    new_im.save(fileName)
    df.to_excel(os.path.basename(imagePath)[:-4] + "_" + str(randrange(100)) + ".xlsx")
    
    img = Image.open(fileName)
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype(fontPath, 16)
    # draw.text((x, y),"Sample Text",(r,g,b))
    draw.text((2330, 827),"Sample Text",(255,255,255),font=font)
    draw.text((2330, 4281),"Sample Text",(255,255,255),font=font)
    draw.text((2330, 4135),"Sample Text",(255,255,255),font=font)
    draw.text((2330, 5789),"Sample Text",(255,255,255),font=font)
    draw.text((4660, 827),"Sample Text",(255,255,255),font=font)
    draw.text((4660, 4281),"Sample Text",(255,255,255),font=font)
    draw.text((4660, 4135),"Sample Text",(255,255,255),font=font)
    draw.text((4660, 5789),"Sample Text",(255,255,255),font=font)    
    
    img.save(fileName)

In [6]:
def createExcelData(imagePath, document):
    datatoExcel = []
    with io.open(imagePath, 'rb') as image_file: #opening and reading img
        content = image_file.read() 
    image = vision.types.Image(content=content) 
    response2 = client.label_detection(image=image)
    for annotation in response2.label_annotations:
        #print(annotation.description + "\t-->\t" + str(annotation.score)) #MID is a knowlege graph
        datatoExcel.append([annotation.description, annotation.score*100])
    response4 = client.document_text_detection(image = image)
    for i in response4.full_text_annotation.pages:
        widthForEntireDocument = i.width
        heightForEntireDocument = i.height
        #print("WIDTH: " + str(i.width))
        #print("HEIGHT: " + str(i.height))
    datatoExcel.append(["WIDTH", widthForEntireDocument])
    datatoExcel.append(["HEIGHT", heightForEntireDocument])
    langConfid = []
    for i in response4.full_text_annotation.pages:
        #print(i.property.detected_languages)
        for lang in i.property.detected_languages:
            currentLangConfid = []
            currentLangConfid.append(lang.language_code)
            #print(lang.confidence)
            currentLangConfid.append(lang.confidence)
            langConfid.append(currentLangConfid)
    for i in langConfid:
        datatoExcel.append([i[0], i[1]*100])
    boundBoxAll = getBoundingBoxes(imagePath, document)
    datatoExcel.append(["Blocks", len(boundBoxAll[0])])
    datatoExcel.append(["Paragraghs", len(boundBoxAll[1])])
    datatoExcel.append(["Words", len(boundBoxAll[2])])
    return pd.DataFrame(datatoExcel, columns=['Info', 'Value'])    

In [7]:
class FeatureType(Enum):
    PAGE = 1
    BLOCK = 2
    PARA = 3
    WORD = 4
    SYMBOL = 5
def get_document_bounds(image_file, feature, document):
    """Returns document bounds given an image."""
    #Using API from Google
    #Returns a JSON file but text is extracted from it
    bounds = []
    # Collect specified feature bounds by enumerating all document features
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    for symbol in word.symbols:
                        if (feature == FeatureType.SYMBOL):
                            bounds.append(symbol.bounding_box)
                    if (feature == FeatureType.WORD):
                        bounds.append(word.bounding_box)
                if (feature == FeatureType.PARA):
                    bounds.append(paragraph.bounding_box)
            if (feature == FeatureType.BLOCK):
                bounds.append(block.bounding_box)
 
    return bounds
def getBoundingBoxes(filein, document):
    image = Image.open(filein)
    
    allb = []
    boundsB = get_document_bounds(filein, FeatureType.BLOCK, document)
    allb.append(boundsB)
    boundsPara = get_document_bounds(filein, FeatureType.PARA, document)
    allb.append(boundsPara)
    boundsWord = get_document_bounds(filein, FeatureType.WORD, document)
    allb.append(boundsWord)
    return allb

In [8]:
def draw_boxes(image, bounds, color):
    """Draw a border around the image using the hints in the vector list."""
    draw = ImageDraw.Draw(image)
    for bound in bounds:
        draw.polygon([
            bound.vertices[0].x, bound.vertices[0].y,
            bound.vertices[1].x, bound.vertices[1].y,
            bound.vertices[2].x, bound.vertices[2].y,
            bound.vertices[3].x, bound.vertices[3].y], None, color)
    return image
def render_doc_text(filein):
    image = Image.open(filein)
    bounds = get_document_bounds(filein, FeatureType.BLOCK, document)
    draw_boxes(image, bounds, 'blue')
    bounds = get_document_bounds(filein, FeatureType.PARA, document)
    draw_boxes(image, bounds, 'red')
    bounds = get_document_bounds(filein, FeatureType.WORD, document)
    draw_boxes(image, bounds, 'yellow')
    #if fileout != 0:
        #image.save(fileout)
    #else:
    image.show()

In [9]:
def findArea(bounds):
    matrix = ((bounds.vertices[0].x, bounds.vertices[0].y),
              (bounds.vertices[1].x, bounds.vertices[1].y),
              (bounds.vertices[2].x, bounds.vertices[2].y),
              (bounds.vertices[3].x, bounds.vertices[3].y))    
    polygon = Polygon(matrix)
    return polygon.area

In [10]:
def draw_single_box(image, bounds):
    """Draw a border around the image using the hints in the vector list."""
    draw = ImageDraw.Draw(image)

    draw.polygon([
        bounds.vertices[0].x, bounds.vertices[0].y,
        bounds.vertices[1].x, bounds.vertices[1].y,
        bounds.vertices[2].x, bounds.vertices[2].y,
        bounds.vertices[3].x, bounds.vertices[3].y],outline=(0,0,0,255))
    return image

In [11]:
def drawsingleBoxVertice(imagefile, color, transp, vertices):
    bounds = vertices
    matrix = ((bounds.vertices[0].x, bounds.vertices[0].y),
              (bounds.vertices[1].x, bounds.vertices[1].y),
              (bounds.vertices[2].x, bounds.vertices[2].y),
              (bounds.vertices[3].x, bounds.vertices[3].y))    
    TINT_COLOR = color
    TRANSPARENCY = transp
    OPACITY = int(255 * TRANSPARENCY)
    img = Image.open(imagefile)
    img = img.convert("RGBA")
    overlay = Image.new('RGBA', img.size, TINT_COLOR+(0,))
    draw = ImageDraw.Draw(overlay)  # Create a context for drawing things on it.
    draw.polygon(matrix, fill=TINT_COLOR+(OPACITY,))
    img = Image.alpha_composite(img, overlay)
    img = img.convert("RGB") # Remove alpha for saving in jpg format.
    return img

In [12]:
def findMatrix(vert):
    matrix = ((vert.vertices[0].x, vert.vertices[0].y),
              (vert.vertices[1].x, vert.vertices[1].y),
              (vert.vertices[2].x, vert.vertices[2].y),
              (vert.vertices[3].x, vert.vertices[3].y))
    return matrix

In [13]:
def colorALL(image_file, boundBoxAll):
    TINT_COLOR = (0,0,0) 
    #colors = [(0,255,0), (0,0,255), (255,0,0)]
    TRANSPARENCY = 0.25
    OPACITY = int(255 * TRANSPARENCY)
    img = Image.open(image_file)
    img = img.convert("RGBA")
    overlay = Image.new('RGBA', img.size, TINT_COLOR+(0,))
    draw = ImageDraw.Draw(overlay)  # Create a context for drawing things on it.
    #for section in boundBoxAll:
    TINT_COLOR = (0,255,0) 
    for block in boundBoxAll[0]:
        matrix = findMatrix(block)
        draw.polygon(matrix, fill=TINT_COLOR+(OPACITY,))
    TINT_COLOR = (0,0,255)
    for para in boundBoxAll[1]:
        matrix = findMatrix(para)
        draw.polygon(matrix, fill=TINT_COLOR+(OPACITY,))
    TINT_COLOR = (255,0,0)
    for word in boundBoxAll[2]:
        matrix = findMatrix(word)
        draw.polygon(matrix, fill=TINT_COLOR+(OPACITY,))
    img = Image.alpha_composite(img, overlay)
    img = img.convert("RGB") # Remove alpha for saving in jpg format.
    return img

In [14]:
def colorAllSkills(imagefile, listofSkills, document):
    allWords = []
    boxesForSkills = []
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    wordRun = ""
                    for symbol in word.symbols:
                        wordRun+=symbol.text
                    allWords.append(wordRun)
                    if wordRun in listofSkills:
                        boxesForSkills.append(word.bounding_box)
    TINT_COLOR = (0,0,0) 
    #colors = [(0,255,0), (0,0,255), (255,0,0)]
    TRANSPARENCY = 0.25
    OPACITY = int(255 * TRANSPARENCY)
    img = Image.open(imagefile)
    img = img.convert("RGBA")
    overlay = Image.new('RGBA', img.size, TINT_COLOR+(0,))
    draw = ImageDraw.Draw(overlay)  # Create a context for drawing things on it.
    #for section in boundBoxAll:
    TINT_COLOR = (0,255,0) 
    for para in boxesForSkills:
        matrix = findMatrix(para)
        draw.polygon(matrix, fill=TINT_COLOR+(OPACITY,))
    img = Image.alpha_composite(img, overlay)
    img = img.convert("RGB") # Remove alpha for saving in jpg format.
    return img

In [15]:
def findRandomWordsFromDocument(document):
    allWords = []
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    wordRun = ""
                    for symbol in word.symbols:
                        wordRun+=symbol.text
                    allWords.append(wordRun)
    return random.sample(allWords,15)

In [16]:
def automaticColoringOfEntitiesDocument(imagePath, resumePDFPath, document):
    data = ResumeParser(resumePDFPath).get_extracted_data()
    allWordsTotal = []
    for key in data:
        if data[key] is None:
            pass
        else:
            allWordsTotal.append([data[key], key])
    allWords = []
    boxesTotal = []
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    wordRun = ""
                    for symbol in word.symbols:
                        wordRun+=symbol.text
                    allWords.append(wordRun)
                    for textext in allWordsTotal:
                        if type(textext[0]) == str:
                            if textext[0] == wordRun:
                                #print(textext[1] + wordRun)
                                boxesTotal.append([word.bounding_box, textext[1]])
                                #GOT IT
                            else:
                                for wordsingle in textext[0].split():
                                    if wordsingle == wordRun:
                                        #print(textext[1] + wordRun)   
                                        boxesTotal.append([word.bounding_box, textext[1]])
                        elif type(textext[0]) == list:
                            for skillSingle in textext[0]:
                                if skillSingle == wordRun:
                                    #print(textext[1] + wordRun)
                                    boxesTotal.append([word.bounding_box, textext[1]])
                                else:
                                    for wordsingle in skillSingle.split():
                                        if wordsingle == wordRun:
                                            #print(textext[1] + wordRun)   
                                            boxesTotal.append([word.bounding_box, textext[1]])
                    #if wordRun in listofSkills:
                        #boxesForSkills.append(word.bounding_box)
    colors = [(0,255,0), (0,0,255), (255,0,0), (255,255,0)]
    TINT_COLOR = (0,0,0) 
    TRANSPARENCY = 0.25
    OPACITY = int(255 * TRANSPARENCY)
    img = Image.open(imagePath)
    img = img.convert("RGBA")
    overlay = Image.new('RGBA', img.size, TINT_COLOR+(0,))
    draw = ImageDraw.Draw(overlay)  # Create a context for drawing things on it.
    font = ImageFont.truetype(fontPath, size=20)
    #for section in boundBoxAll:
    for para in boxesTotal:
        matrix = findMatrix(para[0])
        if para[1] == "name":
             TINT_COLOR = colors[0]
        if para[1] == "skills":
             TINT_COLOR = colors[1]
        if para[1] == "mobile_number":
             TINT_COLOR = colors[2]
        if para[1] == "email":
             TINT_COLOR = colors[3]
        draw.polygon(matrix, fill=TINT_COLOR+(OPACITY,))
        textLocation = (matrix[0][0], matrix[0][1]-20)
        (x, y) = textLocation
        message = para[1]
        color = 'rgb(0, 0, 0)' # black color
        draw.text((x, y), message, fill=color, font=font)
    img = Image.alpha_composite(img, overlay)
    img = img.convert("RGB") # Remove alpha for saving in jpg format.
    return img

In [17]:
def detectDate(text):
    doc = nlp(text)
    for entity in doc.ents:
        if entity.label_=="DATE":
            return True
    return False
def detectDateWord(text):
    doc = nlp(text)
    for entity in doc.ents:
        if entity.label_=="DATE":
            return entity.text.split()
    return []

In [18]:
def SpacyDatesColor(document, imagePath):
    dateBoxes = []
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                paraRun = ""
                for word in paragraph.words:
                    wordRun = ""
                    for symbol in word.symbols:
                        wordRun+=symbol.text
                    paraRun+=wordRun+" "

                if detectDate(paraRun.strip()):
                    dateBoxes.append(paragraph.bounding_box)

    color = (0,0,255)
    TINT_COLOR = (0,0,0) 
    TRANSPARENCY = 0.25
    OPACITY = int(255 * TRANSPARENCY)
    img = Image.open(imagePath)
    img = img.convert("RGBA")
    overlay = Image.new('RGBA', img.size, TINT_COLOR+(0,))
    draw = ImageDraw.Draw(overlay)  # Create a context for drawing things on it.
    font = ImageFont.truetype(fontPath, size=20)
    #for section in boundBoxAll:
    TINT_COLOR = color
    for para in dateBoxes:
        matrix = findMatrix(para)
        textLocation = (matrix[0][0], matrix[0][1]-20)
        (x, y) = textLocation
        message = "Date"
        color = 'rgb(0, 0, 0)' # black color
        draw.text((x, y), message, fill=color, font=font)
        draw.polygon(matrix, fill=TINT_COLOR+(OPACITY,))
    for para in dateBoxes:
        matrix = findMatrix(para)
        textLocation = (matrix[0][0], matrix[0][1]-20)
        (x, y) = textLocation
        message = "Date"
        color = 'rgb(0, 0, 0)' # black color
        draw.text((x, y), message, fill=color, font=font)
        draw.polygon(matrix, fill=TINT_COLOR+(OPACITY,))
    img = Image.alpha_composite(img, overlay)
    img = img.convert("RGB") # Remove alpha for saving in jpg format.
    return img

In [19]:
def detect_Maximum_outlier(data_1):
    #z = (X — μ) / σ
    #Formula for Z score = (Observation — Mean)/Standard Deviation
    outliers=[]
    threshold=3
    mean_1 = np.mean(data_1)
    std_1 =np.std(data_1)
    for y in data_1:
        z_score= (y - mean_1)/std_1 
        if np.abs(z_score) > threshold and y > mean_1:
            outliers.append(y)
    return outliers
def findMaxOutliersIQR(datasetInput):
    dataset = sorted(datasetInput)
    q1, q3= np.percentile(dataset,[25,75])
    iqr = q3 - q1
    upper_bound = q3 +(1.5 * iqr) 
    outliers=[]
    for num in dataset:
        if num>upper_bound:
            outliers.append(num)
    return outliers
def averageOfBothOutliers(data):
    return (min(detect_Maximum_outlier(data)) + min(findMaxOutliersIQR(data)))/2

In [20]:
def highlightBigWords(document, imagePath):
    areaSymbol = []
    areaWord = []
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    for symbol in word.symbols:
                        #print(symbol.text+" " +str(findArea(symbol.bounding_box)))
                        areaSymbol.append(findArea(symbol.bounding_box))
                    areaWord.append(findArea(word.bounding_box))
    boundingBoxForLarge = []
    #symbolOutilerAverage = min(findMaxOutliersIQR(areaSymbol))averageOfBothOutliers(data)
    #symbolOutilerAverage = averageOfBothOutliers(areaSymbol)
    symbOutilAve = averageOfBothOutliers(areaSymbol)
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    runningWord = ""
                    listofSymbolsAreas = []
                    for symbol in word.symbols:
                        #print(symbol.text+" " +str(findArea(symbol.bounding_box)))
                        if findArea(symbol.bounding_box)>symbOutilAve:
                            #print(symbol.text)
                            countSymbolBig = True
                            listofSymbolsAreas.append(findArea(symbol.bounding_box))
                        else:
                            countSymbolBig = False
                        runningWord+=symbol.text
                    if countSymbolBig and sum(listofSymbolsAreas)/len(listofSymbolsAreas) > symbOutilAve:
                        #print(runningWord + "\t" +  str(findArea(word.bounding_box)))
                        #print(sum(listofSymbolsAreas)/len(listofSymbolsAreas))
                        boundingBoxForLarge.append(word.bounding_box)
    color = (0,0,255)
    TINT_COLOR = (0,0,0) 
    TRANSPARENCY = 0.25
    OPACITY = int(255 * TRANSPARENCY)
    img = Image.open(imagePath)
    img = img.convert("RGBA")
    overlay = Image.new('RGBA', img.size, TINT_COLOR+(0,))
    draw = ImageDraw.Draw(overlay)  # Create a context for drawing things on it.
    font = ImageFont.truetype(fontPath, size=20)
    #for section in boundBoxAll:
    TINT_COLOR = color
    for large in boundingBoxForLarge:
        matrix = findMatrix(large)
        textLocation = (matrix[0][0], matrix[0][1]-20)
        (x, y) = textLocation
        message = "BigWord"
        color = 'rgb(0, 0, 0)' # black color
        draw.text((x, y), message, fill=color, font=font)
        draw.polygon(matrix, fill=TINT_COLOR+(OPACITY,))
    img = Image.alpha_composite(img, overlay)
    img = img.convert("RGB") # Remove alpha for saving in jpg format.
    return img

In [21]:
def capitalLargeWords(document, imagePath):
    lsttester = []
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    runningWord = ""
                    uppercaseLetterCt = 0
                    totalSymbolsLeterCt = 0
                    for symbol in word.symbols:
                        if symbol.text.isupper():
                            uppercaseLetterCt+=1
                        totalSymbolsLeterCt+=1
                        runningWord+=symbol.text
                    if uppercaseLetterCt == totalSymbolsLeterCt:
                        lsttester.append(findArea(word.bounding_box))
    try:
        thresholdAreaCapitailWord = min(findMaxOutliersIQR(lsttester))                 
    except Exception as ValueError:
        thresholdAreaCapitailWord = max(lsttester)            
    boundingBoxForLargeCapital = []
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    runningWord = ""
                    uppercaseLetterCt = 0
                    totalSymbolsLeterCt = 0
                    for symbol in word.symbols:
                        if symbol.text.isupper():
                            uppercaseLetterCt+=1
                        totalSymbolsLeterCt+=1
                        runningWord+=symbol.text
                    if uppercaseLetterCt == totalSymbolsLeterCt and findArea(word.bounding_box) > thresholdAreaCapitailWord:
                        #print(runningWord)
                        #print(findArea(word.bounding_box))
                        boundingBoxForLargeCapital.append(word.bounding_box)
    color = (0,0,255)
    TINT_COLOR = (0,0,0) 
    TRANSPARENCY = 0.25
    OPACITY = int(255 * TRANSPARENCY)
    img = Image.open(imagePath)
    img = img.convert("RGBA")
    overlay = Image.new('RGBA', img.size, TINT_COLOR+(0,))
    draw = ImageDraw.Draw(overlay)  # Create a context for drawing things on it.
    font = ImageFont.truetype(fontPath, size=20)
    #for section in boundBoxAll:
    TINT_COLOR = color
    for large in boundingBoxForLargeCapital:
        matrix = findMatrix(large)
        textLocation = (matrix[0][0], matrix[0][1]-20)
        (x, y) = textLocation
        message = "BigCapitalWord"
        color = 'rgb(0, 0, 0)' # black color
        draw.text((x, y), message, fill=color, font=font)
        draw.polygon(matrix, fill=TINT_COLOR+(OPACITY,))
    img = Image.alpha_composite(img, overlay)
    img = img.convert("RGB") # Remove alpha for saving in jpg format.
    return img

# Running

In [10]:
#Using API from Google
#Returns a JSON file but text is extracted from it
keyDIR = imgTxtVisionAPIPath #JSON key file to call the api
credentials = service_account.Credentials.from_service_account_file(keyDIR) #using service account to go through google
client = vision.ImageAnnotatorClient(credentials=credentials) # client api

In [11]:
for i in os.listdir(docPath): 
    if i.endswith(".pdf"):
        print(i)

Document_1504.pdf
Document_502.pdf
Document_608.pdf
Document_866.pdf
Document_882.pdf


In [12]:
#Makes a folder and create images of each page in PDF document
textList = []
numberPagesList = []
pathsforAllImages = []
for i in os.listdir(docPath): 
    if i.endswith(".pdf"):# only runs the file that was choosen by user
        newPathGroupforPDF = []
        os.chdir(docPath) 
        folderNameCreate = docPath+"Fold_"+i[:-4]+"/" 
        print(folderNameCreate)
        os.mkdir(folderNameCreate) #Create folder
        shutil.move(i,folderNameCreate) #MovePDF into folder
        os.chdir(folderNameCreate)
        for j in os.listdir(folderNameCreate):
            print(j)
            if j.endswith(".pdf"): 
                imgName = convert_pdf_2_image(folderNameCreate, j) #call pdf2img function
                for x in imgName:
                    newPathGroupforPDF.append([folderNameCreate+x, folderNameCreate+i])
                print("Running Number: " + str(len(imgName))) 
                printString = ""
                for docName in imgName: # for making the printing better
                    printString += docName + " " 
                print(printString) 
        pathsforAllImages.append(newPathGroupforPDF)

/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_1504/
Document_1504.pdf
Running Number: 2
Document_1504_1.jpg Document_1504_2.jpg 
/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_502/
Document_502.pdf
Running Number: 1
Document_502_1.jpg 
/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_608/
Document_608.pdf
Running Number: 3
Document_608_1.jpg Document_608_2.jpg Document_608_3.jpg 
/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_866/
Document_866.pdf
Running Number: 2
Document_866_1.jpg Document_866_2.jpg 
/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_882/
Document_882.pdf
Running Number: 3
Document_882_1.jpg Document_882_2.jpg Document_882_3.jpg 


In [13]:
pathsforAllImages

[[['/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_1504/Document_1504_1.jpg',
   '/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_1504/Document_1504.pdf'],
  ['/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_1504/Document_1504_2.jpg',
   '/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_1504/Document_1504.pdf']],
 [['/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_502/Document_502_1.jpg',
   '/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_502/Document_502.pdf']],
 [['/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_608/Document_608_1.jpg',
   '/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_608/Document_608.pdf'],
  ['/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_608/Document_608_2.jpg',
   '/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_608/Document_608.pdf'],
  ['/Users/kunal/Documents/VdartResumeProject

In [32]:
for r in pathsforAllImages:
    for group in r:
        collectAll(fontPath, group[0], group[1], finalSaveImg)

C:\Users\kunal\Anaconda3\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.1). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
===

# Other Graphs

In [ ]:
def fig2img(fig):
    """Convert a Matplotlib figure to a PIL Image and return it"""
    buf = io.BytesIO()
    fig.savefig(buf)
    buf.seek(0)
    img = Image.open(buf)
    return img

In [ ]:
def fig2data (fig):
    fig.canvas.draw ( )
    # Get the RGBA buffer from the figure
    w,h = fig.canvas.get_width_height()
    buf = numpy.fromstring ( fig.canvas.tostring_argb(), dtype=numpy.uint8 )
    buf.shape = ( w, h,4 )
    # canvas.tostring_argb give pixmap in ARGB mode. Roll the ALPHA channel to have it in RGBA mode
    buf = numpy.roll ( buf, 3, axis = 2 )
    return buf

In [ ]:
def fig2img(fig):
    # put the figure pixmap into a numpy array
    buf = fig2data ( fig )
    w, h, d = buf.shape
    return Image.fromstring( "RGBA", ( w ,h ), buf.tostring( ) )

In [ ]:
fig = plt.hist(areaSymbol, bins = 25)
#plt.title("Area of Characters")

In [ ]:
plt.style.use('ggplot')
(n, bins, patches) = plt.hist(areaSymbol, bins = 25)
plt.title("Area of Characters")
plt.show()
res = statistics.median(areaSymbol)
print("Normal size of character {}".format(res))
print("Total Number of characters {}".format(len(areaSymbol)))

In [ ]:
plt.style.use('ggplot')
plt.hist(areaWord, bins = 20)
plt.title("Area of Words")
plt.show()
res = statistics.median(areaWord)
print("Normal size of character {}".format(res))
print("Total Number of characters {}".format(len(areaWord)))